In [1]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [190]:
%sendtofile main.py --source main.py

Sent 63 lines (1230 bytes) to main.py.


In [91]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.envh")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

connecting to network...
network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')


In [98]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

decode_period_s = 4 # decode every x seconds
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]
api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot')

runner = OnlineRunner('CCA', buffer_size=Ns)
runner.setup(**log_params)
runner.run()

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
connecting to network...
network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')


In [100]:
print(runner.output_buffer)
runner.stop()

[-0.04179526, -1.87689, -6.050216, -4.619785, -12.30089, -29.34657, -5.430662, 16.15306, 1.520026, 17.09361, 36.15711, 20.44824, -9.487555, -29.90106, -20.17232, -15.53988, -22.40484, 13.78118, 37.3049, 35.42534, 45.44753, 18.2136, 4.41837, -17.95101, -44.72243, -27.52366, -11.02454, 18.07533, 25.26217, 21.53173, 16.21344, 14.81441, -3.934866, -38.38978, -21.07615, 0.900435, 3.538435, -31.65422, -29.26047, -6.156304, 8.204639, 39.88416, 14.10834, 10.32538, 8.813786, -14.11746, -33.01486, -17.81416, 1.329798, -19.52213, 19.01212, 37.36596, 21.30258, 0.3455695, -27.02314, -29.83115, -23.26448, -1.170373, 21.85099, 28.86534, 6.050815, -15.00704, -33.18097, -20.95881, -0.03471351, -0.9796466, -0.3909593, 4.687751, -6.90556, -29.54284, -27.91251, -14.46558, 14.10185, 32.66275, 16.51373, 7.94457, 6.191947, -12.45231, -38.09886, -15.3043, -3.268488, -18.47307, 16.48532, 36.10726, 20.96221, -2.681967, -26.20954, -33.31726, -22.65532, -3.912046, 21.35466, 27.39192, 4.310492, -15.41527, -35.6973

In [78]:
f = open('lib/.envh', 'r')
print(f.read())

#.envh
WIFI_SSID=Rishil
WIFI_PASSWORD=rishilhotspot



In [134]:
%sendtofile main.py --source main.py

Sent 348 lines (6514 bytes) to main.py.


# Local functionality

In [2]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(calibration_data[7])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(calibration_data[10])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)
print(calibration_data)

....1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.....array([[0.03755076, 2.363869, 9.336431, ..., -17.92521, 5.644498, 6.648876],
       [-0.03196555, -2.372048, -13.33754, ..., 8.940619, 13.82679, 18.16005],
       [-0.006059172, -1.555875, -12.42068, ..., 3.434669, 14.42593, 9.65546],
       [-0.02817595, -2.395233, -14.50669, ..., 10.48451, 11.47818, 11.543]], dtype=float32)
42400
64576
.....array([[-0.02973128, -1.71763, -4.537876, ..., 5.599152, 14.15059, 11.52598],
       [0.003306907, -1.19471, -11.81768, ..., 14.94073, -18.18897, -16.96204],
       [0.05126918, 2.216113, 6.343579, ..., 19.09368, 19.97264, 20.38114],
       [-0.01140821, 0.6448166, 6.833508, ..., 20.34509, 19.20622, 16.56798]], dtype=float32)
38256
60416
.....34480
51968
...3
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
4
5
.Traceback (most recent call last):
  File "lib/runner.py", line 144, in sample_callback
  File "lib/runner.py", line 109, in decode
  F

       [-0.03196555, -2.372048, -13.33754, ..., 8.940619, 13.82679, 18.16005],
       [-0.006059172, -1.555875, -12.42068, ..., 3.434669, 14.42593, 9.65546],
       [-0.02817595, -2.395233, -14.50669, ..., 10.48451, 11.47818, 11.543]], dtype=float32)}


In [5]:
print(calibration_data)

{12: array([[0.05357344, 5.530156, 28.47921, ..., -79.63006, -51.04057, -8.808574],
       [-0.08614445, -5.620097, -19.46981, ..., -32.20666, 25.38081, 98.05659],
       [-0.05272939, -4.329994, -15.65205, ..., 51.08261, -64.73856, -63.94084],
       [0.09854264, 4.762796, 12.37467, ..., -56.98923, -56.32615, 19.33137]], dtype=float32), 10: array([[0.08359065, 2.295247, -1.706748, ..., -36.67161, 69.65841, 67.08278],
       [0.0882, 2.69173, -0.008597493, ..., -13.44658, -71.08087, -14.23057],
       [0.06483975, 6.278597, 30.48612, ..., 37.12779, 90.10909, 45.15176],
       [-0.05932719, -5.334993, -23.36372, ..., -64.19778, -55.48473, -67.24873]], dtype=float32), 7: array([[0.1005572, 3.724821, 5.96211, ..., -12.49524, -31.71693, 13.52216],
       [-0.07552055, -3.18233, -4.498146, ..., 84.21824, 5.221718, -30.90145],
       [0.09351017, 5.90537, 21.14749, ..., -75.68513, 21.45787, 59.54765],
       [-0.05849295, -1.497095, -6.886073, ..., -28.30564, -44.03179, 31.94718]], dtype=flo

# Local with downsample

In [9]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(calibration_data[7])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(calibration_data[10])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

decode = Runner('MsetCCA', buffer_size=256) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

....1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.

In [41]:
print(calibration_data[7][0])
print(calibration_data[7][1])
print(calibration_data[7][2])

array([-0.02948636, -17.76074, 9.155772, ..., 3.700044, -15.78295, 8.143198], dtype=float32)
array([-0.02370476, -15.8685, 14.00164, ..., 11.1347, -8.946104, 14.10908], dtype=float32)
array([-0.03774101, 4.773279, 3.103825, ..., 8.202699, -11.71021, 12.6524], dtype=float32)


In [126]:
decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
4
5


In [127]:
for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

.{12: 0.11574, 10: 0.1195, 7: 0.01818}
.{12: 0.02805, 10: 0.05897, 7: 0.01001}
.{12: 0.01652, 10: 0.08894, 7: 0.02123}
{12: 0.06124, 10: 0.04174, 7: 0.04992}
.{12: 0.0727, 10: 0.10921, 7: 0.02819}
6
7


# Online Functionality

In [201]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['ets Jul 29 2019 12:21:46']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:5204']
[leftinbuffer] ['load:0x40078000,len:12136']
[leftinbuffer] ['load:0x40080400,len:3496']
[leftinbuffer] ['entry 0x4008063c']
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [198]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

# env_vars = load_env_vars("lib/.envh")
# # connect WiFI
# ssid = env_vars.get("WIFI_SSID")
# password = env_vars.get("WIFI_PASSWORD")
# ssid ="Rishil"
# password = "rishilhotspot"
# connect_wifi(ssid, password)

# # import webrepl
# # webrepl.start()

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration
api_host = "http://192.168.0.13:5001/"
log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot_test_wireless')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()


...

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 54, in <module>
KeyboardInterrupt: 


In [195]:
runner.stop()

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, i


[missing-OK]  File "lib/runner.py", line 109, in decode

[missing-OK]  File "lib/decoding.py", line 237, in classify

[missing-OK]  File "lib/decoding.py", line 41, in compute_corr

[missing-OK]  File "lib/decoding.py", line 145, in cca_eig

[missing-OK]ValueError: dimensions do not match

[missing-OK]Traceback (most recent call last):

[missing-OK]  File "lib/runner.py", line 144, in sample_callback

[missing-OK]  File "lib/runner.py", line 109, in decode

[missing-OK]  File "lib/decoding.py", line 237, in classify

[missing-OK]  File "lib/decoding.py", line 41, in compute_corr

[missing-OK]  File "lib/decoding.py", line 145, in cca_eig

[missing-OK]ValueError: dimensions do not match

[missing-OK]Traceback (most recent call last):

[missing-OK]  File "lib/runner.py", line 144, in sample_callback

[missing-OK]  File "lib/runner.py", line 109, in decode

[missing-OK]  File "lib/decoding.py", line 237, in classify

[missing-OK]  File "lib/decoding.py", line 41, in compute_corr

[missin

In [173]:
print(runner.decoded_output)

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, i

# Online with downsample

In [45]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['ets Jul 29 2019 12:21:46']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:5204']
[leftinbuffer] ['load:0x40078000,len:12136']
[leftinbuffer] ['load:0x40080400,len:3496']
[leftinbuffer] ['entry 0x4008063c']
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [47]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration
api_host = "http://192.168.0.13:5001/"

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='downsample_test')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()


....1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.....61168
65600
.....60096
64528
.....59008
63440
...3
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
connecting to network...
.network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [40]:
runner.calibrate(calibration_data)

In [52]:
runner.stop()

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 380, in OpenRequestJSONData']
[leftinbuffer] ['Exception: Error to convert object to JSON format']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 380, in OpenRequestJSONData']
[leftinbuffer] ['Exception: Error to convert object to JSON format']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 380, in OpenRequestJSONData']
[leftinbuffer] ['Exception: Error to convert object to JSON format']
[

In [51]:
print(runner.decoder.decoder_stack[12].Y)

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 380, in OpenRequestJSONData']
[leftinbuffer] ['Exception: Error to convert object to JSON format']
array([[0.0001697876, -0.02068205, -0.01098429, ..., 0.08806051, -0.06752927, 0.03739241],
       [-7.910204e-05, 0.03849339, -0.003116021, ..., 0.06650053, -0.04979036, 0.03129825],
       [0.000115422, -0.07172754, 0.08438979, ..., 0.09504408, -0.04703465, 0.03734086],
       [-2.254332e-05, -0.07254709, 0.07210047, ..., -0.04729412, 0.02752842, -0.05740129]], dtype=float32)


# Data send

In [22]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ["b'\\r\\nBro\\x81d\\xfd\\xfcets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nload:0x3fff0018,len:4\\r\\nload:0x3fff001c,len:5204\\r\\nload:0x40078000,len:12136\\r\\nload:0x40080400,len:3496\\r\\nentry 0x4008063c\\r\\n\\x00ets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nload:0x3fff0018,len:4\\r\\nload:0x3fff001c,len:5204\\r\\nload:0x40078000,len:12136\\r\\nload:0x40080400,len:3496\\r\\nentry 0x4008063c\\r\\n\\r\\nBrownout detector was triggered\\r\\n\\r\\nets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0xc (SW_CPU_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00

In [96]:
%sendtofile main.py --source main.py

Sent 156 lines (3317 bytes) to main.py.


In [89]:
%sendtofile lib/decoding.py --source lib/decoding.py

Sent 240 lines (8891 bytes) to lib/decoding.py.


In [205]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

# counter = 0
# while counter !=10:
#     if (counter % 2 == 0):
#         p26.on()
#         p13.off()
#     else:
#         p13.on()
#         p26.off()
#     time.sleep(1)
#     counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 3

api_host = "http://192.168.0.13:5001/lol"

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='ssvep')

runner = OnlineRunner('CCA', buffer_size=Ns)
runner.setup(**log_params)

runner.run()

..ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [206]:
runner.stop()

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 381, in OpenRequestJSONData']
[leftinbuffer] ['  File "lib/requests.py", line 282, in OpenRequest']
[leftinbuffer] ['Exception: Error to connect to 192.168.0.13:5001']


In [162]:
print(runner.output )

from .peripherals import PeripheralManager
from ulab import numpy as np
from logging import BaseLogger, MQTTLogger, HTTPLogger, logger_types

import gc
import config

from micropython import schedule


class Runner:
    def __init__(self, decoding_algo, buffer_size=256, stimulus_freqs=None) -> None:
        if stimulus_freqs is None:
            self.stim_freqs = config.STIM_FREQS  # assign defaults

        self.decoding_algo = decoding_algo

        self.base_sample_freq = config.ADC_SAMPLE_FREQ
        self.downsampled_freq = config.DOWNSAMPLED_FREQ

        self.preprocessing_enabled = config.PREPROCESSING

        if self.preprocessing_enabled:
            self.downsampled_freq = config.DOWNSAMPLED_FREQ

        self.sample_counter = 0
        self.buffer_size = buffer_size  # TODO: update this to be populated dynamically

        self.output_buffer = [0.0 for i in range(self.buffer_size)]
        self.decoded_output = {}

        self.is_setup = False
        self.is_sampling = F

# sending calibration data with posts

Potentially could be the decoding on the server side not working, maybe bypass this by sending it array by array, but this didn;t work last time I tried.

In [93]:
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [6]:
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [74]:
payload = {"b":[123]}#json.dumps({"a":[123]})
a = requests.JSONRequest("http://192.168.0.13:5001/", payload)
b = requests.JSONRequest("http://192.168.0.13:5001/calibration",payload)
print(a)
print(b)

.{'msg': 'data stored successfully'}
{'msg': 'successfully received'}


In [94]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

# time.sleep(5)

# p26.off()
# p13.off()

# time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner  


def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return data

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s))
        
    runner.stop()

    gc.collect()
    p26.off()
    return trials

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup()

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

calibration_data[7] = getData(Nt, decode_period_s)
toSend = {"7": calibration_data[7]}
requests.JSONRequest("http://192.168.0.13:5001/7hz", toSend)
del calibration_data[7]
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print("=====================================================")
toSend = {"10": calibration_data[10]}
requests.JSONRequest("http://192.168.0.13:5001/10hz", toSend)
del calibration_data[10]
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
toSend = {"12": calibration_data[12]}
requests.JSONRequest("http://192.168.0.13:5001/12hz", toSend)
del calibration_data
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
print(3)
runner.run()

while True:
    time.sleep(4)
    data = preprocess_data(runner.output_buffer)
    toSend = {"raw_data_downsampled":data}
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)

1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.connecting to network...
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
....43120
50224
......=====================================================
31504
49728
.....34384
50800
.3
...............................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 156, in <module>
  File "lib/requests.py", line 163, in JSONRequest
  File "lib/requests.py", line 381, in OpenRequestJSONData
  File "lib/requests.py", line 262, in OpenRequest
  File "lib/requests.py", line 445, in URL
  File "lib/requests.py", line 86, in _urlEncode
  File "lib/requests.py", line 76, in _quote
KeyboardInterrupt: 


In [80]:
x = requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
print(x)

b'calibrated'


In [95]:
runner.stop()

In [81]:
data = preprocess_data(runner.output_buffer)
print(data)
toSend = {"raw_data_downsampled":data}
requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)

[-0.1265608, 14.75374, 13.01991, 223.3412, -73.86499, 31.23314, 78.07077, -9.998867, -88.18929, 122.8474, -6.213243, -11.48255, -104.5924, 1.624012, 82.55065, -26.98083, 0.08283337, -30.13198, -30.18656, 68.6827, 4.596216, 78.22694, -0.8851838, 25.04312, 55.96723, 87.47169, -50.18561, -19.24967, 41.73672, -71.67779, -74.2983, 70.4039, -0.09571329, 55.03326, 3.600224, -48.3371, -97.15296, 91.90772, -90.69921, -116.122, 35.48123, 52.91813, -2.816526, 19.02113, 76.16733, 36.37492, -40.99196, 43.79368, -0.1013191, 59.91276, 70.8144, -76.39969, -79.6292, 108.1437, -48.39102, -56.38085, -76.66, 77.36151, 42.77882, -104.7679, 40.76835, 113.1776, -11.24875, -25.63907]
.

In [37]:
runner.stop()

In [ ]:
def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)